In [11]:
import numpy as np 
from random import randint
import matplotlib.pyplot as plt 
import itertools 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import os.path
from pylab import savefig 
import copy

#Model Assumptions
#Fitness values range from 0 to 1 (standard assumption)
#Semi-Random seeding of initial pop (Try rand value, if that square is occupied, just take the first open square)
#Birth and Death rate the same and fixed for all cell types
#No mutation (as of yet)
#Should prolif afford a fitness benefit 
#The can't push layer of cells implies no brth in interaction step
#doesnt consider resources 

In [12]:
#Set Universal Parameters 
K = 5

In [13]:
#List of random grid indicies 
def CAInds():
    CAind = []
    perm = itertools.permutations(np.arange(0,K),2) 

    for i in np.arange(0,K):
        CAind.append((i,i))
    for i in perm:
        CAind.append(i)
        
    np.random.shuffle(CAind)

    return(CAind)


In [14]:
#Neighborhood Function 

def GetNeigh(i):
    data = {'0':[i[0],i[1]+1],'1':[i[0]-1,i[1]+1],'2':[i[0]-1,i[1]],'3':[i[0]-1,i[1]-1],'4':[i[0],i[1]-1],'5':[i[0]+1,i[1]-1],'6':[i[0]+1,i[1]],'7':[i[0]+1,i[1]-1]}
    n = randint(0,7)
    neigh = data[str(n)]
    return(neigh)

In [15]:
def REdge(i):
    data = {'0':[i[0]-1,i[1]],'1':[i[0]-1,i[1]-1],'2':[i[0],i[1]-1],'3':[i[0]+1,i[1]-1],'4':[i[0]+1,i[1]]}
    n = randint(0,4)
    neigh = data[str(n)]
    return(neigh)


In [16]:
def Top(i):
    data = {'0':[i[0],i[1]+1],'1':[i[0]+1,i[1]+1],'2':[i[0]+1,i[1]],'3':[i[0]+1,i[1]-1],'4':[i[0],i[1]-1]}
    n = randint(0,4)
    neigh = data[str(n)]
    return(neigh)
    

In [17]:
def LEdge(i):
    data = {'0':[i[0]-1,i[1]],'1':[i[0]-1,i[1]+1],'2':[i[0],i[1]+1],'3':[i[0]+1,i[1]+1],'4':[i[0]+1,i[1]]}
    n = randint(0,4)
    neigh = data[str(n)]
    return(neigh)

In [18]:
def Bottom(i):
    data = {'0':[i[0],i[1]+1],'1':[i[0]-1,i[1]+1],'2':[i[0]-1,i[1]],'3':[i[0]-1,i[1]-1],'4':[i[0],i[1]-1]}
    n = randint(0,4)
    neigh = data[str(n)]
    return(neigh)

In [19]:
def RTCorner(i):
    data = {'0':[i[0],i[1]-1],'1':[i[0]+1,i[1]-1],'2':[i[0]+1,i[1]]}
    n = randint(0,2)
    neigh = data[str(n)]
    return(neigh)

In [20]:
def LTCorner(i):
    data = {'0':[i[0],i[1]+1],'1':[i[0]+1,i[1]+1],'2':[i[0]+1,i[1]]}
    n = randint(0,2)
    neigh = data[str(n)]
    return(neigh)

In [21]:
def RBCorner(i):
    data = {'0':[i[0]-1,i[1]],'1':[i[0]-1,i[1]-1],'2':[i[0],i[1]-1]}
    n = randint(0,2)
    neigh = data[str(n)]
    return(neigh)

In [22]:
def LBCorner(i):
    data = {'0':[i[0]-1,i[1]],'1':[i[0]-1,i[1]+1],'2':[i[0],i[1]+1]}
    n = randint(0,2)
    neigh = data[str(n)]
    return(neigh)

In [23]:
def CDeath(i,Cfit,d,grid):
    if Cfit[i[0]*K+i[1]] < d:
        grid[i] = 0
        Cfit[i[0]*K+i[1]] = 0 
    return grid,Cfit

In [24]:
def DDeath(i,Dfit,d,grid):
    if Dfit[i[0]*K+i[1]] < d:
        grid[i] = 0
        Dfit[i[0]*K+i[1]] = 0 
    return grid,Dfit

In [25]:
def CBirth(i,neigh_ind,Cfit,b,grid):
    if Cfit[i[0]*K+i[1]]>b:
        Cfit[neigh_ind[0]*K+neigh_ind[1]] = Cfit[i[0]*K+i[1]]
        grid[neigh_ind[0],neigh_ind[1]] = 1
    return grid,Cfit

In [26]:
def DBirth(i,neigh_ind,Dfit,b,grid):
    if Dfit[i[0]*K+i[1]]>b:
        Dfit[neigh_ind[0]*K+neigh_ind[1]] = Dfit[i[0]*K+i[1]]
        grid[neigh_ind[0],neigh_ind[1]] = 2
    return grid,Dfit

In [27]:
#Sets Mutation Threshold 
def Mutate(m,i,grid):
    u = randint(0,10)
    if grid[i] == 1:
        if u>m:
            grid[i] == 2
    elif grid[i] == 2:
        if u>m:
            grid[i] == 1
    return grid 

In [28]:
#Randomly seeds the grid with an initial population of cells
#Ones represent cooperators, twos are defectors, zeros are empty cells
def SeedPop(C0,D0,Cfit0,Dfit0):
    grid = np.zeros((K,K))
    Cfit = np.repeat(0,K*K) 
    Dfit = np.repeat(0,K*K)
    ind = []
    count = 0
    while count < C0+D0:
        a = randint(0,K-1)
        b = randint(0,K-1)
        c = randint(0,K-1)
        d = randint(0,K-1)

        if grid[a,b] == 0:
            grid[a,b] = 1
            Cfit[a*K+b] = Cfit0

        else:
            for i in np.arange(0,K):
                for j in np.arange(0,K):
                    if grid[i,j] ==0:
                        ind.append([i,j])
            grid[ind[0][0],ind[0][1]] = 1
            Cfit[ind[0][0]*K+ind[0][1]] = Cfit0
        count+=1 
        ind = []

        if grid[c,d] == 0:
            grid[c,d] = 2
            Dfit[c*K+d] = Dfit0

        else:
            for i in np.arange(0,K):
                for j in np.arange(0,K):
                    if grid[i,j] ==0:
                        ind.append([i,j])
                        grid[ind[0][0],ind[0][1]] = 2
                        Dfit[ind[0][0]*K+ind[0][1]] = Dfit0
        count+=1
        ind = []
    return grid, Cfit, Dfit

SeedPop(C0=2,D0=2,Cfit0=10,Dfit0=10)

(array([[0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 1., 0., 2., 0.],
        [0., 0., 0., 0., 0.],
        [2., 0., 0., 0., 0.]]),
 array([ 0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0]),
 array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,
         0,  0,  0, 10,  0,  0,  0,  0]))

In [29]:
#Cellular Automata 
#b is the birth rate
#d is the death rate 
#m is the mutation rate, can turn on or off
#ben is the benefit all cells get for interacting with a cooperator 
#cost is the cost of being a cooperator, ie everytime a cooperator interacts (shares resources) a cost is incurred 
#mutation can either be on or off, controls if cell types can mutate into each other 
#years is the number of generations the simulation is run for 
def RunCA(b,d,m,ben,cost,mutation,years):
    gen = 0
    A = SeedPop(C0=2,D0=2,Cfit0=10,Dfit0=10)
    grid = A[0]
    Cfit = A[1]
    Dfit = A[2]
    while gen<years:
        CAind = CAInds()
        Grid[gen]=copy.deepcopy(grid)
        for i in CAind:
            if grid[i] == 1: 
                if i[0]!= 0 and i[1]!= K-1 and i[0]!= K-1 and i[1]!= 0:
                    neigh_ind = GetNeigh(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]

                    if neigh == 1:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]+(ben-cost)
                        CDeath(i,Cfit,d,grid)

                    elif neigh == 2:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]-cost
                        CDeath(i,Cfit,d,grid)

                    elif neigh == 0:
                        CBirth(i,neigh_ind,Cfit,b,grid)
                        CDeath(i,Cfit,d,grid)

                elif i[1]==K-1 and i[0]!=0 and i[0]!=K-1: #checks right edge 
                    neigh_ind = REdge(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]+(ben-cost)
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 2:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]-cost
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 0:
                        CBirth(i,neigh_ind,Cfit,b,grid)
                        CDeath(i,Cfit,d,grid)

                elif i[0] ==0 and i[1]!=K-1 and i[1]!= 0: #checks top 
                    neigh_ind = Top(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]+(ben-cost)
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 2:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]-cost
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 0:
                        CBirth(i,neigh_ind,Cfit,b,grid)
                        CDeath(i,Cfit,d,grid)

                elif i[1] == 0 and i[0]!=0 and i[0]!= K-1: #checks left edge 
                    neigh_ind = LEdge(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]+(ben-cost)
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 2:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]-cost
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 0:
                        CBirth(i,neigh_ind,Cfit,b,grid)
                        CDeath(i,Cfit,d,grid)

                elif i[0] == K-1 and i[1]!= K-1 and i[1]!= 0: #checks bottom 
                    neigh_ind = Bottom(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]+(ben-cost)
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 2:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]-cost
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 0:
                        CBirth(i,neigh_ind,Cfit,b,grid)
                        CDeath(i,Cfit,d,grid)

                elif i[0]==0 and i[1]==K-1: #checks right top corner
                    neigh_ind = RTCorner(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]+(ben-cost)
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 2:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]-cost
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 0:
                        CBirth(i,neigh_ind,Cfit,b,grid)
                        CDeath(i,Cfit,d,grid)

                elif i[0] ==0 and i[1] ==0: #check left top corner
                    neigh_ind = LTCorner(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]+(ben-cost)
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 2:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]-cost
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 0:
                        CBirth(i,neigh_ind,Cfit,b,grid)
                        CDeath(i,Cfit,d,grid)

                elif i[0]==K-1 and i[1]==0: #checks left bottom corner
                    neigh_ind = LBCorner(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]+(ben-cost)
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 2:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]-cost
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 0:
                        CBirth(i,neigh_ind,Cfit,b,grid)
                        CDeath(i,Cfit,d,grid)

                elif i[0]==K-1 and i[1]==K-1: #checks right bottom corner
                    neigh_ind = RBCorner(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]+(ben-cost)
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 2:
                        Cfit[i[0]*K+i[1]] = Cfit[i[0]*K+i[1]]-cost
                        CDeath(i,Cfit,d,grid)
                    elif neigh == 0:
                        CBirth(i,neigh_ind,Cfit,b,grid)
                        CDeath(i,Cfit,d,grid)

            elif grid[i] == 2:
                if i[0]!= 0 and i[1]!= K-1 and i[0]!= K-1 and i[1]!= 0:
                    neigh_ind = GetNeigh(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]

                    if neigh == 1:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]+ben
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 2:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 0:
                        DBirth(i,neigh_ind,Dfit,b,grid)
                        DDeath(i,Dfit,d,grid)

                elif i[1]==K-1 and i[0]!=0 and i[0]!=K-1: #checks right edge 
                    neigh_ind = REdge(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]+ben
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 2:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 0:
                        DBirth(i,neigh_ind,Dfit,b,grid)
                        DDeath(i,Dfit,d,grid)


                elif i[0] ==0 and i[1]!=K-1 and i[1]!= 0: #checks top 
                    neigh_ind = Top(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]+ben
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 2:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 0:
                        DBirth(i,neigh_ind,Dfit,b,grid)
                        DDeath(i,Dfit,d,grid)


                elif i[1] == 0 and i[0]!=0 and i[0]!= K-1: #checks left edge 
                    neigh_ind = LEdge(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]+ben
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 2:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 0:
                        DBirth(i,neigh_ind,Dfit,b,grid)
                        DDeath(i,Dfit,d,grid)


                elif i[0] == K-1 and i[1]!= K-1 and i[1]!= 0: #checks bottom 
                    neigh_ind = Bottom(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]+ben
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 2:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 0:
                        DBirth(i,neigh_ind,Dfit,b,grid)
                        DDeath(i,Dfit,d,grid)


                elif i[0]==0 and i[1]==K-1: #checks right top corner
                    neigh_ind = RTCorner(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]+ben
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 2:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 0:
                        DBirth(i,neigh_ind,Dfit,b,grid)
                        DDeath(i,Dfit,d,grid)

                elif i[0] ==0 and i[1] ==0: #check left top corner
                    neigh_ind = LTCorner(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]+ben
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 2:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 0:
                        DBirth(i,neigh_ind,Dfit,b,grid)
                        DDeath(i,Dfit,d,grid)

                elif i[0]==K-1 and i[1]==0: #checks left bottom corner
                    neigh_ind = LBCorner(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]+ben
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 2:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 0:
                        DBirth(i,neigh_ind,Dfit,b,grid)
                        DDeath(i,Dfit,d,grid)

                elif i[0]==K-1 and i[1]==K-1: #checks right bottom corner
                    neigh_ind = RBCorner(i)
                    neigh = grid[neigh_ind[0],neigh_ind[1]]
                    if neigh == 1:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]+ben
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 2:
                        Dfit[i[0]*K+i[1]] = Dfit[i[0]*K+i[1]]
                        DDeath(i,Dfit,d,grid)

                    elif neigh == 0:
                        DBirth(i,neigh_ind,Dfit,b,grid)
                        DDeath(i,Dfit,d,grid)

                if mutation=='on':
                    Mutate(m,i,grid)
                    
        gen += 1

In [1]:
#Initial Parameters to Use for Part b) 
Grid = np.repeat(0,10) #sets size of grid 
RunCA(b=1,d=0,ben=0,cost=0, m = 0, mutation = 'on',years=5)

NameError: name 'np' is not defined

In [ ]:
#Outputs the Grid 
outpath = '/Users/juliapelesko/Desktop/PHYS320figs' #change to where you want to save the figures on your computer 
def heatmaps():
    count = 0
    for i in moo:
        myfig = sns.heatmap(i,0,2,cmap="inferno")
        fig = myfig.get_figure()
        fig.savefig( outpath + "/CAheatmap" + str(count)+".png")
        plt.clf()
        count+=1
heatmaps()

In [72]:
grid = SeedPop(C0=1,D0=0,Cfit0=10,Dfit0=10)
print(grid)

def RunCA(d,b,ben,cost):
    for i in CAInds():
        print(grid[i])  
    return 0
RunCA(1,2,3,4)

(array([[0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2.]]), array([ 0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0]), array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, 10]))


TypeError: tuple indices must be integers or slices, not tuple

In [69]:
#Spatially Biased CA 
grid = SeedPop(C0=1,D0=0,Cfit0=10,Dfit0=10)
print(grid)


def SpatialCA(b,d,ben,cost):
    for i in np.arange(0,K): 
        for j in np.arange(0,K): 

            #Update fitness based on Payoff Matrix 

            #Cooperators 
            if grid[i,j]==1: 
                #C interact with C 
                if j+1<K and grid[i,j+1]==1:
                    Cfit[i*K+j] = Cfit[i*K+j]+(ben-cost)

                    if Cfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Cfit[i*K+j] = 0

                elif j+1==K and i+1 < K and grid[i+1,0]==1:
                    Cfit[i*K+j] = Cfit[i*K+j]+(ben-cost)

                    if Cfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Cfit[i*K+j] = 0

                elif j+1==K and i+1==K and grid[0,0]==1:
                    Cfit[i*K+j] = Cfit[i*K+j]+(ben-cost)

                    if Cfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Cfit[i*K+j] = 0

                #C Interact with D        
                elif j+1<K and grid[i,j+1]==2:
                    Cfit[i*K+j] = Cfit[i*K+j]-cost

                    if Cfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Cfit[i*K+j] = 0

                elif j+1==K and i+1<K and grid[i+1,0] ==2:
                    Cfit[i*K+j] = Cfit[i*K+j]-cost

                    if Cfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Cfit[i*K+j] = 0

                elif j+1==K and i+1==K and grid[0,0] ==2:
                    Cfit[i*K+j] = Cfit[i*K+j]-cost

                    if Cfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Cfit[i*K+j] = 0

                #Check Birth Thresholds 
                if Cfit[i*K+j] > b and j+1<K and grid[i,j+1] == 0:
                    grid[i,j+1] = 1
                    Cfit[i*K+j+1] = Cfit[i*K+j]

                elif Cfit[i*K+j] > b and j+1==K and i+1<K and grid[i+1,0]==0:
                    grid[i+1,0] = 1 
                    Cfit[i*K+j+1] = Cfit[i*K+j]

                elif j+1==K and i+1==K and Cfit[i*K+j] > b and grid[0,0]==0:
                    grid[0,0] = 1 
                    Cfit[0] = Cfit[i*K+j]
                    
            #Defectors 
            elif grid[i,j]==2: 
                #D interact with D
                if j+1<K and grid[i,j+1]==2: 
                    Dfit[i*K+j] = Dfit[i*K+j]

                    if Dfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Dfit[i*K+j] = 0

                elif j+1==K and i+1 < K and grid[i+1,0]==2:
                    Dfit[i*K+j] = Dfit[i*K+j]

                    if Dfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Dfit[i*K+j] = 0

                elif j+1==K and i+1==K and grid[0,0]==2:
                    Dfit[i*K+j] = Dfit[i*K+j]

                    if Dfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Dfit[i*K+j] = 0

                #D Interact with C     
                elif j+1<K and grid[i,j+1]==1:
                    Dfit[i*K+j] = Dfit[i*K+j]+ben

                    if Dfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Dfit[i*K+j] = 0

                elif j+1==K and i+1<K and grid[i+1,0] ==1:
                    Dfit[i*K+j] = Dfit[i*K+j]+ben

                    if Dfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Dfit[i*K+j] = 0

                elif j+1==K and i+1==K and grid[0,0] ==1:
                    Dfit[i*K+j] = Dfit[i*K+j]+ben

                    if Dfit[i*K+j] < d: 
                        grid[i,j] = 0 
                        Dfit[i*K+j] = 0

                #Check Birth Thresholds       
                if Dfit[i*K+j] > b and j+1<K and grid[i,j+1] == 0:
                    grid[i,j+1] = 2
                    Dfit[i*K+j+1] = Dfit[i*K+j]

                elif j+1==K and i+1<K and Dfit[i*K+j] > b and grid[i+1,0]==0:
                    grid[i+1,0] = 2
                    Dfit[i*K+j+1] = Dfit[i*K+j]

                elif j+1==K and i+1==K and Dfit[i*K+j] > b and grid[0,0]==0:
                    grid[0,0] = 2
                    Dfit[0] = Dfit[i*K+j]

    return grid
RunCA(b=3,d=1,ben=5,cost=1)

(array([[0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0.]]), array([ 0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0]), array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0, 10,  0,  0,  0]))


TypeError: tuple indices must be integers or slices, not tuple

In [26]:
list(perm)


TypeError: 'itertools.permutations' object is not callable

In [29]:
perm